In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from sklearn.utils import shuffle
from keras import optimizers


In [ ]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
swim_list = ['Process_data_freestylestagnant_tag1_chunk1.csv', 'Process_data_freestylestagnant_tag1_chunk2.csv', 'Process_data_freestylestagnant_tag1_chunk3.csv', 'Process_data_freestylestagnant_tag1_chunk4.csv', 'Process_data_freestylestagnant_tag2_chunk1.csv', 'Process_data_freestylestagnant_tag2_chunk2.csv', 'Process_data_freestylestagnant_tag2_chunk3.csv', 'Process_data_freestylestagnant_tag2_chunk4.csv']

In [ ]:
drown_list = ['Process_data_drowning1_tag1_chunk0.csv','Process_data_drowning1_tag1_chunk1.csv', 'Process_data_drowning1_tag2_chunk0.csv', 'Process_data_drowning1_tag2_chunk1 .csv', 'Process_data_drowning2_tag1_chunk0.csv', 'Process_data_drowning3_tag1_chunk0.csv', 'Process_data_drowning3_tag1_chunk1.csv', 'Process_data_drowning3_tag1_chunk2.csv', 'Process_data_drowning3_tag2_chunk0.csv', 'Process_data_drowning3_tag2_chunk1.csv']

In [ ]:
len(drown_list)

In [ ]:
sequences = list()
swim_train = []
swim_validate = []
swim_test = []

In [ ]:
for file in enumerate(swim_list):
    if file[0]<18:
        swim_train.append(file[1])
    elif file[0]<24:
        swim_validate.append(file[1])
    else:
        swim_test.append(file[1])

In [ ]:
swim_train_list =np.empty((0,1000),int)
swim_validate_list =np.empty((0,1000),int)
swim_test_list = np.empty((0,1000),int)

In [ ]:
for file in swim_train:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_train_list=np.append(swim_train_list,values.reshape(1,1000),axis=0)


In [ ]:
for file in swim_validate:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_validate_list=np.append(swim_validate_list,values.reshape(1,1000),axis=0)

for file in swim_test:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_test_list=np.append(swim_test_list,values.reshape(1,1000),axis=0)


####Drown Data####

In [ ]:

drown_train =[]
drown_validate =[]
drown_test =[]

In [ ]:
for file in enumerate(drown_list):
    if file[0]<6:
        drown_train.append(file[1])
    elif file[0]<8:
        drown_validate.append(file[1])
    else:
        drown_test.append(file[1])

In [ ]:
for file in drown_train:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_train_list=np.append(swim_train_list,values.reshape(1,1000),axis=0)
    
for file in drown_validate:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_validate_list=np.append(swim_validate_list,values.reshape(1,1000),axis=0)

for file in drown_test:
    df = np.genfromtxt(file,delimiter=',',skip_header=1)
    values = df[:,2].reshape(1,1000)    
    swim_test_list=np.append(swim_test_list,values.reshape(1,1000),axis=0)

In [ ]:
swim_train_list = np.array(swim_train_list)
swim_validate_list = np.array(swim_validate_list)
swim_test_list = np.array(swim_test_list)

In [ ]:
swim_train_list.shape

In [ ]:
train_target_swim = np.zeros(18)
train_target_drown = np.ones(6)
train_target = np.concatenate([train_target_swim,train_target_drown])

validate_target_swim = np.zeros(6)
validate_target_drown = np.ones(2)
validate_target = np.concatenate([validate_target_swim,validate_target_drown])

test_target_swim = np.zeros(6)
test_target_drown = np.ones(2)
test_target = np.concatenate([test_target_swim,test_target_drown])

In [ ]:
swim_train_list, train_target = shuffle(swim_train_list, train_target, random_state=0)
swim_validate_list, validate_target = shuffle(swim_validate_list, validate_target, random_state=0)
swim_test_list, test_target = shuffle(swim_test_list, test_target, random_state=0)

In [ ]:
swim_train_list.shape
train_target

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(1000,1)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
adam = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(swim_train_list, train_target, epochs=20, batch_size=64, validation_data=(swim_validate_list,validate_target))

In [ ]:
from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(swim_test_list)
accuracy_score(test_target, test_preds)

In [ ]:
test_preds

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
scores = {}
scores_list = []
for k in range(1,25):
    knn = KNeighborsClassifier(n_neighbors=k, p=2)
    knn.fit(swim_train_list,train_target)
    y_pred=knn.predict(swim_test_list)
    scores[k] = metrics.accuracy_score(test_target,y_pred)
    scores_list.append(metrics.accuracy_score(test_target,y_pred))

In [ ]:
scores

In [ ]:
scores_list

In [ ]:
from sklearn.svm import SVC  
svclassifier = SVC(kernel='rbf', C=0.001, gamma=0.001)  
svclassifier.fit(swim_train_list, train_target)  

In [ ]:
y_pred = svclassifier.predict(swim_test_list)  

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(train_target,y_pred))  
print(classification_report(train_target,y_pred))  

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_target, y_pred, sample_weight=None)

In [ ]:
y_pred

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
svc_param_selection(swim_train_list, train_target, 10)